# 2.2 Individual Model 2

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)
device

device(type='cpu')

In [2]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0

import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
batch_size = 128

In [3]:
def print_settings(winmin,stridesec, EPOCHS):
    outfile = sys.stdout

    winlength = int(winmin * 60 * 15)
    step = int(stridesec * 15)
    start_time = datetime.now()
    arr = ["echo -n 'PBS: node is '; cat $PBS_NODEFILE",\
          "echo PBS: job identifier is $PBS_JOBID",\
          "echo PBS: job name is $PBS_JOBNAME"]

    [os.system(cmd) for cmd in arr]
    print("*****************************************************************\n", file=outfile, flush=True)
    print("Execution Started at " + start_time.strftime("%m/%d/%Y, %H:%M:%S"), file=outfile, flush=True)
    print("WindowLength: {:.2f} min ({:d} datum)\tSlide: {:d} ({:d} datum)\tEpochs:{:d}\n".format(winmin, winlength, stridesec, step, EPOCHS), file=outfile, flush=True)


In [4]:
EPOCHS = 30
winmin = 1 
stridesec = 1

print_settings(winmin,stridesec, EPOCHS)
# Load the dataset
meal_data_train = Person_MealsDataset(person_name= "lawler", file_name = "train_files", winmin = winmin,stridesec = stridesec)
meal_data_test = Person_MealsDataset(person_name= "lawler", file_name = "test_files", winmin = winmin,stridesec = stridesec)


*****************************************************************

Execution Started at 12/18/2020, 00:17:50
WindowLength: 1.00 min (900 datum)	Slide: 1 (15 datum)	Epochs:30

Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.19/10.19.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/afternoon_2hr33min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/evening_2hr_20min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.21/10.21.shm
Loading File:  ../data/IndividualData/lawler-data/10.26/10.26.shm
Loading File:  ../data/IndividualData/lawler-data/10.28/10.28.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/10.31.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/evening-2hr_goodDinnerTemplate_CFAmeal/10.31.shm
Loading File:  ../data/IndividualData/lawler-data/10.4/10.4.shm
Loading File:  ../data/IndividualData/lawler-data/10.6/10.6.shm
Loading File:  ../data/IndividualData/lawler-data/10.8/10.8.shm
Loading F

In [5]:
#random.seed(random_seed)

# load
trainingsamples,traininglabels =  meal_data_train.data_indices, meal_data_train.labels
testsamples,testlabels =  meal_data_test.data_indices, meal_data_test.labels

train_shuffledUnderSampledBalancedIndices = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed)
test_shuffledUnderSampledBalancedIndices = balance_data_indices(testlabels,mode="under", shuffle=True,random_state = random_seed)



In [6]:

train_set_balanced = torch.utils.data.Subset(meal_data_train, train_shuffledUnderSampledBalancedIndices)
test_set_balanced = torch.utils.data.Subset(meal_data_test, test_shuffledUnderSampledBalancedIndices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set_balanced ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [7]:
# Test to check if random generated indices can be reproducted
# ls = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed)
# ls ==train_shuffledUnderSampledBalancedIndices

In [8]:
from model import *
from utils import *


torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[1], train_set_balanced[0][0].shape[0])
model_4 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1).to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer_4 = optim.Adam(model_4.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_4 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_4, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, test_loader)
model_4, best_model_4,val_score_4,loss_ls_4, train_acc_ls_4, valid_acc_ls_4 = train_model(model_4,dataloader, optimizer_4, 
                                                                    criterion, lrscheduler_4, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/lawler_models/checkpoint_model_resnet.pt" )

Training set batch amounts: 357
Test set : 70
Start Training..
Epoch: 0,  Epoch_Loss: 0.6271, Train Acc: 68.7224 %, Train Recall: 0.6884 
Validation Acc:  62.9306 %,  Validation Recall: 0.5790 
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 0.5568, Train Acc: 72.2921 %, Train Recall: 0.6852 
Validation Acc:  60.7606 %,  Validation Recall: 0.3725 


Epoch: 2,  Epoch_Loss: 0.5497, Train Acc: 72.6315 %, Train Recall: 0.6837 
Validation Acc:  51.3535 %,  Validation Recall: 0.0300 


Epoch: 3,  Epoch_Loss: 0.5552, Train Acc: 72.0030 %, Train Recall: 0.6676 
Validation Acc:  52.5280 %,  Validation Recall: 0.8284 


Epoch: 4,  Epoch_Loss: 0.5523, Train Acc: 72.2154 %, Train Recall: 0.6776 
Validation Acc:  57.6622 %,  Validation Recall: 0.1935 


Epoch: 5,  Epoch_Loss: 0.5547, Train Acc: 72.3118 %, Train Recall: 0.6834 
Validation Acc:  50.1119 %,  Validation Recall: 0.0022 


Epoch: 6,  Epoch_Loss: 0.5605, Train Acc: 71.2934 %, Train Recall: 0.6712 
Validation Acc:  50.0000 %,  Validation Recall:

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/wenkanw/.conda/envs/mlenv/lib/python3.8/multiprocessing/connection.py", line 368, 

KeyboardInterrupt: 

In [ ]:
from model import *
from utils import *


torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[1], train_set_balanced[0][0].shape[0])
model_4 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1).to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer_4 = optim.Adam(model_4.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_4 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_4, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, test_loader)
model_4, best_model_4,val_score_4,loss_ls_4, train_acc_ls_4, valid_acc_ls_4 = train_model(model_4,dataloader, optimizer_4, 
                                                                    criterion, lrscheduler_4, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/wenkanw_models/checkpoint_model_resnet.pt" )